In [4]:
import numpy as np
import pandas as pd
from scipy.optimize import minimize, differential_evolution

# ----------------------------
# Step 1: Load Historical Data
# ----------------------------
# Replace with your own dataset (columns = assets, rows = returns)
# Example: 4 assets (bonds, equities, real estate, commodities)
np.random.seed(42)
n_periods = 1000
n_assets = 4
returns = pd.DataFrame(
    np.random.multivariate_normal(
        mean=[0.05, 0.1, 0.08, 0.06], 
        cov=np.diag([0.02, 0.15, 0.1, 0.12]), 
        size=n_periods
    ),
    columns=["Bonds", "Equities", "RealEstate", "Commodities"]
)

# ----------------------------
# Step 2: Define CVaR Functions
# ----------------------------
alpha = 0.05  # 95% CVaR

def compute_portfolio_returns(weights, returns):
    return returns.dot(weights)

def compute_cvar(portfolio_returns, alpha):
    var = np.quantile(portfolio_returns, alpha)
    cvar = -portfolio_returns[portfolio_returns <= var].mean()
    return cvar

def compute_cvar_contributions(weights, returns, alpha):
    portfolio_returns = compute_portfolio_returns(weights, returns)
    var = np.quantile(portfolio_returns, alpha)
    tail_scenarios = portfolio_returns <= var
    contributions = []
    for i in range(len(weights)):
        asset_contribution = -returns.iloc[:, i][tail_scenarios].mean() * weights[i]
        contributions.append(asset_contribution)
    total_cvar = compute_cvar(portfolio_returns, alpha)
    # Standardize contributions as a percentage of total CVaR
    contributions_pct = [c / total_cvar for c in contributions] if total_cvar != 0 else [0]*len(weights)
    return np.array(contributions_pct)

# ----------------------------
# Step 3: Optimization for MCC Portfolio
# ----------------------------
def objective_mcc(weights, returns, alpha):
    contributions = compute_cvar_contributions(weights, returns, alpha)
    return np.max(contributions)  # Minimize the largest CVaR contribution

# Constraints: sum(weights) = 1, weights >= 0
constraints = (
    {"type": "eq", "fun": lambda w: np.sum(w) - 1},
)
bounds = tuple((0, 1) for _ in range(n_assets))

# Initial guess (equal-weighted)
initial_weights = np.ones(n_assets) / n_assets

# Solve using Sequential Least Squares Programming (SLSQP)
result = minimize(
    fun=objective_mcc,
    x0=initial_weights,
    args=(returns, alpha),
    method="SLSQP",
    bounds=bounds,
    constraints=constraints,
    tol=1e-8
)
mcc_weights = result.x

# ----------------------------
# Step 4: Compare with Other Strategies
# ----------------------------
# Equal-weighted portfolio
ew_weights = np.ones(n_assets) / n_assets

# Minimum CVaR portfolio (minimizes total CVaR)
def objective_min_cvar(weights, returns, alpha):
    portfolio_returns = compute_portfolio_returns(weights, returns)
    return compute_cvar(portfolio_returns, alpha)

result_min_cvar = minimize(
    fun=objective_min_cvar,
    x0=initial_weights,
    args=(returns, alpha),
    method="SLSQP",
    bounds=bounds,
    constraints=constraints,
    tol=1e-8
)
min_cvar_weights = result_min_cvar.x

# ----------------------------
# Step 5: Print Results
# ----------------------------
print("Optimal Weights:")
print(f"- MCC Portfolio: {np.round(mcc_weights, 3)}")
print(f"- Equal-Weighted: {np.round(ew_weights, 3)}")
print(f"- Minimum CVaR: {np.round(min_cvar_weights, 3)}")

# Compute CVaR contributions for each portfolio
def print_contributions(weights, name):
    contributions = compute_cvar_contributions(weights, returns, alpha)
    print(f"\n{name} Portfolio CVaR Contributions (%):")
    for i, asset in enumerate(returns.columns):
        print(f"{asset}: {contributions[i]:.1%}")

print_contributions(mcc_weights, "MCC")
print_contributions(ew_weights, "Equal-Weighted")
print_contributions(min_cvar_weights, "Minimum CVaR")

Optimal Weights:
- MCC Portfolio: [0.218 0.218 0.29  0.274]
- Equal-Weighted: [0.25 0.25 0.25 0.25]
- Minimum CVaR: [0.611 0.108 0.147 0.134]

MCC Portfolio CVaR Contributions (%):
Bonds: -3.0%
Equities: 27.0%
RealEstate: 37.8%
Commodities: 38.2%

Equal-Weighted Portfolio CVaR Contributions (%):
Bonds: -1.7%
Equities: 41.2%
RealEstate: 26.7%
Commodities: 33.8%

Minimum CVaR Portfolio CVaR Contributions (%):
Bonds: 61.7%
Equities: 10.7%
RealEstate: 14.7%
Commodities: 12.9%
